In [ ]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix,classification_report
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df_train=pd.read_csv("/content/train.csv")
df_test=pd.read_csv("/content/test.csv")

In [ ]:
df_train.shape

(5080, 8)

In [ ]:
df_test.shape

(3263, 4)

In [ ]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
y.value_counts()

target
0    2884
1    2196
Name: count, dtype: int64

In [ ]:
df_train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [ ]:
df_train.dropna(inplace=True)

In [ ]:
df_train.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [ ]:
def clean_text(text):

    if not isinstance(text, str):
      text = str(text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text


In [ ]:
df_train['tweet_length']=df_train['text'].apply(len)
df_train['num_words']=df_train['text'].apply(lambda x:len(nltk.word_tokenize(x)))
df_train['num_sentences']=df_train['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df_train[df_train['target']==0][['tweet_length', 'num_words', 'num_sentences']].describe()

,tweet_length,num_words,num_sentences
count,2884.000000,2884.000000,2884.000000
mean,97.607836,19.006935,1.707351
std,35.471422,7.315445,1.130261
min,8.000000,1.000000,1.000000
25%,70.000000,14.000000,1.000000
50%,104.000000,20.000000,1.000000
75%,131.000000,24.000000,2.000000
max,152.000000,70.000000,22.000000


In [ ]:
df_train[df_train['target']==1][['tweet_length', 'num_words', 'num_sentences']].describe()

,tweet_length,num_words,num_sentences
count,2196.000000,2196.000000,2196.000000
mean,108.508197,19.303279,1.431239
std,29.361814,5.942090,0.783839
min,17.000000,3.000000,1.000000
25%,87.000000,15.000000,1.000000
50%,115.000000,20.000000,1.000000
75%,136.000000,24.000000,2.000000
max,148.000000,39.000000,7.000000


In [ ]:
x = df_train['text'] = df_train['text'].apply(clean_text)
x_test = df_test['text'] = df_test['text'].apply(clean_text)
y = df_train['target']

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scl = scaler.fit_transform(df_train[['tweet_length']])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()

X_train_tfidf = tf.fit_transform(x)
X_test_tfidf = tf.transform(x)

# Models:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
svm_classifier = SVC(kernel='linear')

svm_classifier.fit(x_train, y_train)

SVC(kernel='linear')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = svm_classifier.predict(x_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.89      0.83       580
           1       0.82      0.68      0.74       436

    accuracy                           0.80      1016
   macro avg       0.80      0.78      0.79      1016
weighted avg       0.80      0.80      0.79      1016

Confusion Matrix:
[[514  66]
 [141 295]]


In [ ]:
lr_model = LogisticRegression(random_state=42)
lr_model.fit(x_train, y_train)

LogisticRegression(random_state=42)

In [ ]:
y_pred = lr_model.predict(x_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.90      0.83       580
           1       0.83      0.63      0.72       436

    accuracy                           0.79      1016
   macro avg       0.80      0.77      0.77      1016
weighted avg       0.79      0.79      0.78      1016

Confusion Matrix:
[[523  57]
 [161 275]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy',
                                       random_state = 0)
classifier_rf.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [ ]:
y_pred = classifier_rf.predict(x_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.88      0.81       580
           1       0.80      0.62      0.70       436

    accuracy                           0.77      1016
   macro avg       0.77      0.75      0.75      1016
weighted avg       0.77      0.77      0.76      1016

Confusion Matrix:
[[511  69]
 [167 269]]


In [ ]:
from sklearn.naive_bayes import GaussianNB
naive_bayes_classifier = GaussianNB()
naive_bayes_classifier.fit(x_train.toarray(), y_train)

GaussianNB()

In [ ]:
y_pred = naive_bayes_classifier.predict(x_test.toarray())

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.50      0.60       596
           1       0.51      0.73      0.60       420

    accuracy                           0.60      1016
   macro avg       0.62      0.62      0.60      1016
weighted avg       0.64      0.60      0.60      1016

Confusion Matrix:
[[300 296]
 [112 308]]


# Interpretation and Application

The best Model is **svm** based on evaluation metrics

*   Classify social media posts related to disasters
*   Removing non-disaster related posts
*  Highlighting messages with critical details about ongoing disasters so emergency responders can focus on the most important information.

Real-time Disaster Monitoring and Information Dissemination:

* Developer  an API that can process incoming social media data speficly about disaster in real-time, applying the trained model to classify new posts.





